In [3]:
# 라이브러리 불러오기 및 함수화
from collections import Counter
import re
import pandas as pd
from tqdm import tqdm

import torch
from konlpy.tag import Komoran
from pykospacing import Spacing
from transformers import ElectraTokenizer, ElectraModel

komoran = Komoran()
spacing = Spacing()

tqdm.pandas() # tqdm과 pandas 통합

c:\anaconda3\envs\projectM3\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
df = pd.read_csv('./data/combined_questions.csv', encoding='cp949')
# df = df[:][0:100]  # 샘플 100개만

In [ ]:
# 불용어 데이터 불러오기 및 추가

# 기본 불용어 텍스트 파일 경로
korean_stopwords_path = "data/stopwords-ko.txt"

# 기본 불용어 불러오기
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 추가 할 불용어 설정
add_stopwords = [
    
    ]

# 추가 불용어 적용
stopwords.extend(add_stopwords)

In [ ]:
# 띄어쓰기, 대소문자 보정 함수
def preprocessing(text):
    text = spacing(text)
    text = text.lower()                   # 소문자 변경
    text = re.sub(r'[^\w\s]', '', text)
    return text

# komoran토큰화 & 불용어 처리 함수 정의
def remove_stopwords(text, stopwords):
    tokens = []
    morphs = komoran.morphs(text)
    for token in morphs:
        if token not in stopwords:
            tokens.append(token)
    return tokens

In [7]:
# 전처리 데이터를 데이터 프레임에 추가

cleaned_data = []

for i in range(len(df)):
    feature_text = df.loc[i, 'feature']  # 각 열에서 'feature'값 가져오기
    
    processed_text = preprocessing(feature_text)
    cleaned_text = remove_stopwords(processed_text, stopwords)
    cleaned_data.append(cleaned_text)

df['cleaned'] = cleaned_data  # 'cleaned'컬럼 추가해 데이터 삽입

In [8]:
# KoElectra 모델과 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")

# 토큰 벡터화 함수
def text_to_vector(text):
    # 시퀀스 길이를 같게 하기 위해 패딩 추가  #길이가 긴 시퀀스는 잘라냄  # PyTorch 텐서로 반환
    tokenized_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # 모델에 입력하여 출력 벡터 얻기
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
    # [CLS] 토큰에 대한 벡터 추출
    cls_vector = outputs.last_hidden_state[0][0].numpy()
    return cls_vector

df['vector'] = df['cleaned'].apply(text_to_vector)  # 토큰 벡터화하여 'vector'컬럼에 데이터 삽입
df

c:\anaconda3\envs\projectM3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\4호실-3\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\anaconda3\envs\projectM3\Lib\site-packages\transformers\modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=Fal

,feature,target,cleaned,vector
0,2032년 12월 samsung 주가가 어떻게 돼?,stock,"[2032, 12월, samsu, ng, 주가, 어떻, 게, 되]","[-0.22109647, 0.14832048, -0.20444757, -0.0787..."
1,7월 7일 samsung 주가가 어떻게 돼?,stock,"[7월 7일, samsu, ng, 주가, 어떻, 게, 되]","[-0.17597577, -0.0697006, -0.20189564, -0.0971..."
2,5월 월말 samsung 주가가 어떻게 돼?,stock,"[5월, 월말, samsu, ng, 주가, 어떻, 게, 되]","[0.22228521, -0.048565466, -0.22414865, 0.1950..."
3,30일 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[30, 평일, 마지막, 주, samsu, ng, 주가, 어떻, 게, 되]","[0.22256678, 0.050112892, -0.21061091, -0.0129..."
4,2월 금요일 samsung 주가가 어떻게 돼?,stock,"[2월, 금요일, samsung, 주가, 어떻, 게, 되]","[0.18069062, -0.072183356, -0.27727425, 0.1172..."
...,...,...,...,...
95,월초 samsung 주가가 어떻게 돼?,stock,"[월초, samsu, ng, 주가, 어떻, 게, 되]","[-0.10992738, 0.26265565, -0.3298583, -0.03007..."
96,23일 samsung 주가가 어떻게 돼?,stock,"[23, sams, u, ng, 주가, 어떻, 게, 되]","[-0.2622148, 0.019670565, -0.22337738, -0.2043..."
97,11월 평일 마지막주 samsung 주가가 어떻게 돼?,stock,"[11월, 평일, 마지막, 주, samsu, ng, 주가, 어떻, 게, 되]","[0.18449137, 0.00071876775, -0.23439988, 0.083..."
98,9월 월초 마지막주 samsung 주가가 어떻게 돼?,stock,"[9월, 월초, 마지막, 주, samsung, 주가, 어떻, 게, 되]","[0.1705331, -0.028076224, -0.21021225, 0.12850..."
